In [1]:
import re
import os
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
import urllib 

In [2]:
results = []

file = open('D:/Intern_Project/12_medicalxpress/medical_article_info_2/medical_article_info_2.json', 'r', encoding='utf-8')
results = []
for line in file.readlines():
    dic = json.loads(line)
    results.append(dic)

In [3]:
print(len(results))

186408


In [4]:
results[:10]

[{'title': "ChatGPT shows 'impressive' accuracy in clinical decision making",
  'content': '<article class="news-article">\n                                    <div class="row">\n                                        <div class="col-8">\n                                            <div class="article__info mb-2">\n                                                <div class="article__info-item mr-auto">\n                                                    <a href="https://medicalxpress.com/archive/22-08-2023/" rel="nofollow">\n                                                        <svg class="article__info-icon">\n                                                            <use href="https://medx.b-cdn.net/tmpl/v6/img/svg/sprite.svg#icon_clock" x="0" y="0"></use>\n                                                        </svg>\n                                                    </a>\n                                                    <p class="text-uppercase text-low">\n             

In [5]:
def recursive_parser(item, texts, pre_class):
    for child in item:
        try:
#             if isinstance(child, str):
#                 content = child.strip()
#                 if len(content) > 0:
#                     texts.append(content)
#                 continue

            tag = child.name
#             print(tag)
#             print(tag.attrs)
            
            content = child.text.strip()
#             print(content)
            if len(content) == 0:
#                 print("000")
                continue
            
            if tag == 'p' and 'class' in child.attrs:
                class_ = child.attrs['class'][0]
                if class_ in ['article-main__note mt-4']:
                    continue
            
            if tag in ['script', 'link', 'table', 'tbody', 'img', 'iframe', 'em', 'figure', 'aside','iframe', 'html', 'ins']: 
#                 print("111")
                continue

            if tag == 'center':
                texts.append(content)
                continue
            

            if tag == 'div' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['printpreview_header', 'fdaWrapper', 'adCont']:
                    continue
            
            if tag == 'section' and 'id' in child.attrs and len(child.attrs['id']) > 0:
                id_ = child.attrs['id']
                if id_ in ['sponsors']:
                    continue
                    
            if tag == 'section' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                class_ = child.attrs['class'][0]
                if class_ in ['editorial-sources']:
                    continue
            
            if tag == 'div' and 'class' in child.attrs and len(child.attrs['class']) > 0:
                # 对div标签的class进行判别，不同情况分别处理
                class_ = child.attrs['class'][0]
#                 print(class_)
                if class_ in ['SECONDARY', 'AUTHORS', 'HNUM', 'BY', 'SG', 'PN', 'pagenum', 'TB', 'SIDEBAR', 
                              'article-gallery lightGallery', 'w-100 mb-4 d-print-none', 'article-main__more p-4', 'd-inline-block text-medium my-4', 
                              'mt-3'
                             ]:
                    continue

                if class_ in ['h1', 'h2', 'h3', 'h4', 'h5']:
                    texts.append(content)

                elif class_ in ['HD', 'FRTITLE']:
                    texts.append(content)

                elif class_ == 'P':
                    texts.append(content)

                else:
                    recursive_parser(child, texts, class_)
                    
            elif tag in ['section', 'div', 'ul', 'ol']:
                # 这几类标签 直接递归处理子标签
                recursive_parser(child, texts, '')
                
                
            else:
                # 剩余情况直接获取标签内容
                texts.append(content)
                
#             print(texts)
        except Exception as e:
            print(e)

In [6]:
def parser(results, parser_results):
    count = 0
    for record in results:
        
        count += 1
        
        html_content = record['content']
        selector = Selector(text=html_content)
        try:
            title = selector.xpath("//h1//text()").extract()[0].strip()
        except Exception as e:
            print(e)
            continue

        texts = []

        soup = BeautifulSoup(html_content, 'html.parser') 


        item = soup.find('div',{'class':'mt-4 article-main'})

        if item is None:
            print(record['title'])
            continue
        else:
            recursive_parser(item, texts, "")
            content = '\n'.join(texts)
            if content == "":
                continue
            tmp = {
                "title": title,
                "content": content
            }
            # print(tmp)
            parser_results.append(tmp)
        if count % 1000 == 0:
            print(count)

In [7]:
parser_results = []
parser(results, parser_results)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

In [10]:
df= pd.DataFrame(parser_results)

In [12]:
df

,title,content
0,ChatGPT shows 'impressive' accuracy in clinica...,A new study led by investigators from Mass Gen...
1,"Protein score, from a single plasma sample, pr...",In a large retrospective analysis using measur...
2,1 billion people worldwide will suffer from ar...,The degenerative joint disease osteoarthritis ...
3,Police involvement may hamstring overdose outr...,A new study finds law enforcement officials pl...
4,ALS and frontotemporal dementia show origins i...,Even though neurodegenerative diseases often s...
...,...,...
186403,What the US could learn about vaccination from...,"To consider that Nigeria, infamous for anti-va..."
186404,Visible public health leadership needed to boo...,Public health expert Professor John Ashton is ...
186405,Can a business owner require staffers to get v...,Small business owners worried about the spread...
186406,"Vaccination: In Canada, many teenagers don't n...","A few months ago in the United States, Ohio te..."


In [13]:
with open(f'D:/Intern_Project/12_medicalxpress/parser_results1.json', 'r', encoding="utf-8") as f:
    results1 = json.load(f)
df2= pd.DataFrame(results1)
df2

,title,content
0,Study examines role of metabolites in abdomina...,"Each year, about 200,000 people in the United ..."
1,Abdominal aortic aneurysm: New treatment may r...,The intravenous delivery of immune-modulating ...
2,Research guides FDA action on common medical d...,"The largest blood vessel in the body, the abdo..."
3,Algorithm predicts women have higher risk for ...,When receiving treatment for abdominal aortic ...
4,The molecular mechanism behind abdominal aneur...,When the artery that supplies the stomach and ...
...,...,...
57221,"Study shows link between economic, social, env...",More than one in four adults in California rep...
57222,High anxiety levels make patients take more pa...,The greater the level of anxiety when visiting...
57223,Researchers develop foldable tent for safe den...,Dental treatments are performed at close proxi...
57224,"Yes, baby teeth fall out. But they're still im...","Baby teeth, or milk teeth, act like lighthouse..."


In [14]:
parser_results.extend(results1)

In [16]:
df3 = pd.DataFrame(parser_results)
df3

,title,content
0,ChatGPT shows 'impressive' accuracy in clinica...,A new study led by investigators from Mass Gen...
1,"Protein score, from a single plasma sample, pr...",In a large retrospective analysis using measur...
2,1 billion people worldwide will suffer from ar...,The degenerative joint disease osteoarthritis ...
3,Police involvement may hamstring overdose outr...,A new study finds law enforcement officials pl...
4,ALS and frontotemporal dementia show origins i...,Even though neurodegenerative diseases often s...
...,...,...
243629,"Study shows link between economic, social, env...",More than one in four adults in California rep...
243630,High anxiety levels make patients take more pa...,The greater the level of anxiety when visiting...
243631,Researchers develop foldable tent for safe den...,Dental treatments are performed at close proxi...
243632,"Yes, baby teeth fall out. But they're still im...","Baby teeth, or milk teeth, act like lighthouse..."


In [20]:
with open(f'D:/Intern_Project/12_medicalxpress/parser_results_all.json', 'w', encoding="utf-8") as f:
    f.write(json.dumps(parser_results, indent=1, ensure_ascii=False))

MemoryError: 

In [22]:
# 法二：使用流式写入，逐个元素地将数据写入文件，而不需要将整个列表加载到内存中
with open(f'D:/Intern_Project/12_medicalxpress/parser_results_all.json', 'w', encoding="utf-8") as f:
    for item in json.JSONEncoder().iterencode(parser_results):
        f.write(item)

In [23]:
# 法三：使用循环逐个元素地写入文件。这样可以避免一次性加载整个列表到内存中。例如，可以使用json.dump()函数逐个写入列表元素。
with open(f'D:/Intern_Project/12_medicalxpress/parser_results_all_2.json', 'w', encoding="utf-8") as f:
    for item in parser_results:
        json.dump(item, f)
        f.write('\n')  # 添加换行符分隔每个元素

In [19]:
print(len(parser_results))

243634


In [21]:
df3.to_excel("parser_results_all.xlsx")